Завдання 1. Завантажити датасет за посиланням https://www.kaggle.com/datasets/aravindpcoder/obesity-or-cvd-risk-classifyregressorcluster

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
from google.colab import files

# Завантаження файлу
uploaded = files.upload()

Saving ObesityDataSet.csv to ObesityDataSet (1).csv


In [ ]:
# Завантаження CSV-файлу
df = pd.read_csv('ObesityDataSet.csv')

# Перегляд перших 5 рядків
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


Про набір даних Дані складаються з оцінки рівня ожиріння у людей із Мексики, Перу та Колумбії віком від 14 до 61 року та різноманітними харчовими звичками та фізичним станом. Дані були зібрані за допомогою веб-платформи з опитуванням, у якому анонімні користувачі відповідали на кожне запитання. питання, потім інформація була оброблена, отримано 17 атрибутів і 2111 записів. Атрибути, пов’язані зі звичками харчування: часте споживання висококалорійної їжі (FAVC), частота споживання овочів (FCVC), кількість основних прийомів їжі (NCP), споживання їжі між прийомами їжі (CAEC), щоденне споживання води (CH20). ) і споживання алкоголю (CALC). Атрибути, пов’язані з фізичним станом: моніторинг споживання калорій (SCC), частота фізичної активності (FAF), час використання технологічних пристроїв (TUE), використаний транспорт (MTRANS). Отримані змінні: стать, вік, зріст і вага.

Значення NOwesity є:

• Недостатня вага Менше 18,5 • Нормальна 18,5 до 24,9 • Надмірна вага 25,0 до 29,9 • Ожиріння I 30,0 до 34,9 • Ожиріння II 35,0 до 39,9 • Ожиріння III Більше 40

Дані містять числові дані та безперервні дані, тому їх можна використовувати для аналізу на основі алгоритмів класифікації, прогнозування, сегментації та асоціації.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [ ]:
df.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


In [ ]:
df['Gender'].value_counts()

,count
Gender,
Male,1068
Female,1043


In [ ]:
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

In [ ]:
df['MTRANS'].value_counts()

,count
MTRANS,
Public_Transportation,1580
Automobile,457
Walking,56
Motorbike,11
Bike,7


In [ ]:
df['family_history_with_overweight'].value_counts()

,count
family_history_with_overweight,
yes,1726
no,385


In [ ]:
df['family_history_with_overweight'] = df['family_history_with_overweight'].map({'yes': 1, 'no': 0})

In [ ]:
df['CAEC'].value_counts()

,count
CAEC,
Sometimes,1765
Frequently,242
Always,53
no,51


In [ ]:
# Заміна категорій на числові значення
df['CAEC'] = df['CAEC'].map({
    'Sometimes': 1,   # Можна призначити значення відповідно до частоти або важливості
    'Frequently': 2,
    'Always': 3,
    'no': 0
})

In [ ]:
df['FAVC'].value_counts()

,count
FAVC,
yes,1866
no,245


In [ ]:
# Заміна 'yes' на 1 та 'no' на 0
df['FAVC'] = df['FAVC'].map({'yes': 1, 'no': 0})

In [ ]:
# Заміна 'yes' на 1 та 'no' на 0
df['SMOKE'] = df['SMOKE'].map({'yes': 1, 'no': 0})

In [ ]:
df['SCC'].value_counts()

,count
SCC,
no,2015
yes,96


In [ ]:
# Заміна 'yes' на 1 та 'no' на 0
df['SCC'] = df['SCC'].map({'yes': 1, 'no': 0})

In [ ]:
df['CALC'].value_counts()

,count
CALC,


In [ ]:
# Заміна категорій на числові значення
df['CALC'] = df['CALC'].map({
    'Sometimes': 1,  # Можна призначити значення відповідно до частоти
    'no': 0,
    'Frequently': 2,
    'Always': 3
})

In [ ]:
df['MTRANS'].value_counts()

,count
MTRANS,
Public_Transportation,1580
Automobile,457
Walking,56
Motorbike,11
Bike,7


In [ ]:
# Заміна категорій на числові значення
df['MTRANS'] = df['MTRANS'].map({
    'Public_Transportation': 0,
    'Automobile': 1,
    'Walking': 2,
    'Motorbike': 3,
    'Bike': 4
})

In [ ]:
df['NObeyesdad'].value_counts()

,count
NObeyesdad,
Obesity_Type_I,351
Obesity_Type_III,324
Obesity_Type_II,297
Overweight_Level_I,290
Overweight_Level_II,290
Normal_Weight,287
Insufficient_Weight,272


In [ ]:
# Заміна категорій на числові значення
df['NObeyesdad'] = df['NObeyesdad'].map({
    'Obesity_Type_I': 5,
    'Obesity_Type_III': 7,
    'Obesity_Type_II': 6,
    'Overweight_Level_I': 3,
    'Overweight_Level_II': 4,
    'Normal_Weight': 2,
    'Insufficient_Weight': 1
})

In [ ]:
df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,21.000000,1.620000,64.000000,1,0,2.0,3.0,Sometimes,0,2.000000,0,0.000000,1.000000,NaN,0,2
1,0,21.000000,1.520000,56.000000,1,0,3.0,3.0,Sometimes,1,3.000000,1,3.000000,0.000000,NaN,0,2
2,1,23.000000,1.800000,77.000000,1,0,2.0,3.0,Sometimes,0,2.000000,0,2.000000,1.000000,NaN,0,2
3,1,27.000000,1.800000,87.000000,0,0,3.0,3.0,Sometimes,0,2.000000,0,2.000000,0.000000,NaN,2,3
4,1,22.000000,1.780000,89.800000,0,0,2.0,1.0,Sometimes,0,2.000000,0,0.000000,0.000000,NaN,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,20.976842,1.710730,131.408528,1,1,3.0,3.0,Sometimes,0,1.728139,0,1.676269,0.906247,NaN,0,7
2107,0,21.982942,1.748584,133.742943,1,1,3.0,3.0,Sometimes,0,2.005130,0,1.341390,0.599270,NaN,0,7
2108,0,22.524036,1.752206,133.689352,1,1,3.0,3.0,Sometimes,0,2.054193,0,1.414209,0.646288,NaN,0,7
2109,0,24.361936,1.739450,133.346641,1,1,3.0,3.0,Sometimes,0,2.852339,0,1.139107,0.586035,NaN,0,7


In [ ]:
df.duplicated().sum()

24

In [ ]:
df = df.drop_duplicates()
df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,21.000000,1.620000,64.000000,1,0,2.0,3.0,1,0,2.000000,0,0.000000,1.000000,0,0,2
1,0,21.000000,1.520000,56.000000,1,0,3.0,3.0,1,1,3.000000,1,3.000000,0.000000,1,0,2
2,1,23.000000,1.800000,77.000000,1,0,2.0,3.0,1,0,2.000000,0,2.000000,1.000000,2,0,2
3,1,27.000000,1.800000,87.000000,0,0,3.0,3.0,1,0,2.000000,0,2.000000,0.000000,2,2,3
4,1,22.000000,1.780000,89.800000,0,0,2.0,1.0,1,0,2.000000,0,0.000000,0.000000,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,20.976842,1.710730,131.408528,1,1,3.0,3.0,1,0,1.728139,0,1.676269,0.906247,1,0,7
2107,0,21.982942,1.748584,133.742943,1,1,3.0,3.0,1,0,2.005130,0,1.341390,0.599270,1,0,7
2108,0,22.524036,1.752206,133.689352,1,1,3.0,3.0,1,0,2.054193,0,1.414209,0.646288,1,0,7
2109,0,24.361936,1.739450,133.346641,1,1,3.0,3.0,1,0,2.852339,0,1.139107,0.586035,1,0,7


In [ ]:
X = df.drop('NObeyesdad', axis=1)
y = df['NObeyesdad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 3. Побудова моделі логістичної регресії
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)

print("Логістична регресія:")
print(classification_report(y_test, y_pred_log_reg))
print("Точність:", accuracy_score(y_test, y_pred_log_reg))

In [ ]:
# 4. Побудова моделі випадкового лісу
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred_rf = rf_classifier.predict(X_test)
print("\nВипадковий ліс:")
print(classification_report(y_test, y_pred_rf))
print("Точність:", accuracy_score(y_test, y_pred_rf))

In [ ]:
# Масштабування та кодування
# Визначення категоріальних і числових колонок
categorical_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'SCC', 'SMOKE', 'MTRANS']
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Пайплайн для обробки
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Масштабування числових ознак
        ('cat', OneHotEncoder(), categorical_features)  # Одноразове кодування категоріальних ознак
    ])

In [ ]:
# Побудова моделі логістичної регресії
log_reg = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(max_iter=1000))])
log_reg.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'Height', 'Weight',
                                                   'FCVC', 'NCP', 'CH2O', 'FAF',
                                                   'TUE']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Gender',
                                                   'family_history_with_overweight',
                                                   'FAVC', 'SCC', 'SMOKE',
                                                   'MTRANS'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [ ]:
# Побудова моделі RandomForestClassifier
rf_classifier = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))])
rf_classifier.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'Height', 'Weight',
                                                   'FCVC', 'NCP', 'CH2O', 'FAF',
                                                   'TUE']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Gender',
                                                   'family_history_with_overweight',
                                                   'FAVC', 'SCC', 'SMOKE',
                                                   'MTRANS'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
# Оцінка моделей
y_pred_log_reg = log_reg.predict(X_test)
print("Логістична регресія:")
print(classification_report(y_test, y_pred_log_reg))
print("Точність:", accuracy_score(y_test, y_pred_log_reg))

y_pred_rf = rf_classifier.predict(X_test)
print("\nВипадковий ліс:")
print(classification_report(y_test, y_pred_rf))
print("Точність:", accuracy_score(y_test, y_pred_rf))

Логістична регресія:
                     precision    recall  f1-score   support

Insufficient_Weight       0.86      1.00      0.93        56
      Normal_Weight       0.89      0.68      0.77        62
     Obesity_Type_I       0.95      0.90      0.92        78
    Obesity_Type_II       0.90      0.98      0.94        58
   Obesity_Type_III       1.00      1.00      1.00        63
 Overweight_Level_I       0.78      0.75      0.76        56
Overweight_Level_II       0.74      0.84      0.79        50

           accuracy                           0.88       423
          macro avg       0.87      0.88      0.87       423
       weighted avg       0.88      0.88      0.88       423

Точність: 0.8794326241134752

Випадковий ліс:
                     precision    recall  f1-score   support

Insufficient_Weight       0.96      0.96      0.96        56
      Normal_Weight       0.83      0.94      0.88        62
     Obesity_Type_I       0.99      0.94      0.96        78
    Obesity_Ty

Підібрати оптимальні параметри за допомогою GridSearchCV

In [ ]:
# Підбір параметрів для логістичної регресії
#Pipeline — це клас із бібліотеки scikit-learn, що дозволяє послідовно виконувати кілька кроків обробки.

log_reg = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(max_iter=1000))])

# Сітка параметрів для логістичної регресії
log_reg_param_grid = {
    'classifier__C': [0.1, 1, 10, 100],  # Регуляризація
    'classifier__solver': ['liblinear', 'lbfgs']  # Алгоритми оптимізації
}

log_reg_grid_search = GridSearchCV(log_reg, log_reg_param_grid, cv=5, scoring='accuracy')
log_reg_grid_search.fit(X_train, y_train)

# Підбір параметрів для випадкового лісу
rf_classifier = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', RandomForestClassifier(random_state=42))])

# Сітка параметрів для випадкового лісу
rf_param_grid = {
    'classifier__n_estimators': [100, 200, 300],  # Кількість дерев
    'classifier__max_depth': [None, 10, 20, 30],  # Максимальна глибина дерева
    'classifier__min_samples_split': [2, 5, 10],  # Мінімальна кількість зразків для розбиття вузла
    'classifier__min_samples_leaf': [1, 2, 4]  # Мінімальна кількість зразків у листку
}

rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=5, scoring='accuracy')
rf_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'Height',
                                                                          'Weight',
                                                                          'FCVC',
                                                                          'NCP',
                                                                          'CH2O',
                                                                          'FAF',
                                                                          'TUE']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['Gender',
                                                                          'family_history_with_overweight',
                                                                          'FAVC',
                                                                          'SCC',
                                                                          'SMOKE',
                                                                          'MTRANS'])])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             param_grid={'classifier__max_depth': [None, 10, 20, 30],
                         'classifier__min_samples_leaf': [1, 2, 4],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [ ]:
# Оцінка результатів
# Логістична регресія з найкращими параметрами
best_log_reg = log_reg_grid_search.best_estimator_
y_pred_log_reg = best_log_reg.predict(X_test)
print("Логістична регресія з найкращими параметрами:")
print("Найкращі параметри:", log_reg_grid_search.best_params_)
print(classification_report(y_test, y_pred_log_reg))
print("Точність:", accuracy_score(y_test, y_pred_log_reg))

# Випадковий ліс з найкращими параметрами
best_rf = rf_grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_test)
print("\nВипадковий ліс з найкращими параметрами:")
print("Найкращі параметри:", rf_grid_search.best_params_)
print(classification_report(y_test, y_pred_rf))
print("Точність:", accuracy_score(y_test, y_pred_rf))

Логістична регресія з найкращими параметрами:
Найкращі параметри: {'classifier__C': 100, 'classifier__solver': 'lbfgs'}
              precision    recall  f1-score   support

           1       0.95      0.97      0.96        59
           2       0.95      0.93      0.94        61
           3       0.92      0.87      0.90        55
           4       0.88      0.94      0.91        49
           5       1.00      0.96      0.98        70
           6       0.97      1.00      0.98        64
           7       0.98      1.00      0.99        60

    accuracy                           0.95       418
   macro avg       0.95      0.95      0.95       418
weighted avg       0.96      0.95      0.95       418

Точність: 0.9545454545454546

Випадковий ліс з найкращими параметрами:
Найкращі параметри: {'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 300}
              precision    recall  f1-score   support

 

In [ ]:
# Отримання 10 справжніх значень з тестового набору
y_true = y_test[:10].values

# Прогнозування 10 значень за допомогою найкращої моделі логістичної регресії
y_pred_log_reg = best_log_reg.predict(X_test[:10])

# Виведення номера пацієнта, справжніх і прогнозованих значень
print("Номер пацієнта | Справжнє значення | Прогнозоване значення")
for i, index in enumerate(y_test[:10].index):
    print(f"Пацієнт {index} | Справжнє: {y_true[i]} | Прогнозоване: {y_pred_log_reg[i]}")


Номер пацієнта | Справжнє значення | Прогнозоване значення
Пацієнт 1337 | Справжнє: 5 | Прогнозоване: 5
Пацієнт 1763 | Справжнє: 6 | Прогнозоване: 6
Пацієнт 389 | Справжнє: 3 | Прогнозоване: 3
Пацієнт 595 | Справжнє: 1 | Прогнозоване: 1
Пацієнт 1735 | Справжнє: 6 | Прогнозоване: 6
Пацієнт 900 | Справжнє: 3 | Прогнозоване: 3
Пацієнт 243 | Справжнє: 4 | Прогнозоване: 4
Пацієнт 1847 | Справжнє: 7 | Прогнозоване: 7
Пацієнт 624 | Справжнє: 1 | Прогнозоване: 1
Пацієнт 51 | Справжнє: 2 | Прогнозоване: 2
